![](../graphics/microsoftlogo.png)

# Workshop: Unlocking AI Potential for the Data Professional - Azure OpenAI

#### <i>A Microsoft Course from Microsoft Engineering and the FastTrack Team</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

<img style="float: left; margin: 0px 15px 15px 0px;" src="https://raw.githubusercontent.com/microsoft/sqlworkshops/master/graphics/textbubble.png"> <h2>Course Notebook: Module 3</h2>

Welcome to this Microsoft solutions workshop on [*Unlocking AI Potential for the Data Professional with Azure OpenAI*](https://github.com/sqlserverworkshops/OpenAI-DataPro/tree/main). In this Notebook, you'll apply the concepts you learned in this Module.

This Notebook contains recipes for some common applications of machine learning. You'll need a working knowledge of [pandas](http://pandas.pydata.org/), [matplotlib](http://matplotlib.org/), [numpy](http://www.numpy.org/), and, of course, [scikit-learn](http://scikit-learn.org/stable/) to benefit from it.

# 3.1 Create Azure OpenAI Environment

## Azure 

This notebook contains the script to create the necessary Azure environment to run the provided samples. The notebook uses [PowerShell](https://learn.microsoft.com/powershell/scripting/install/installing-powershell?view=powershell-7.3) and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

## Configuration

This notebook performs two tasks:

- Deployment of necessary Azure Services (Azure OpenAI, Azure Cognitive Search) to run samples
- Store all necessary service endpoints, service API keys, Azure OpenAI deployment names in a centralized file (../01_DemoEnvironment/conf/application.env). This file is used by all notebooks in this repo to connect and authenticate against the deployed Azure services.

If you already have instances of Azure OpenAI and Azure Cognitive Search running you can rename the [configuration template](../conf/.env-sample) to `.env` and provide endpoint, API key and deployment names of a chat completion and an embedding model. We suggest to run the notebook to start a clean environment.

### Visual Studio Code

If you are running these steps below in Visual Studio Code make sure you switch your kernal to .NET Interactive so that it will run the PowerShell

## Step 1:   Login to Azure; Get, Set subscription

In [ ]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($null -ne $loggedIn) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}
# Retrieve default subscription id
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set Subscription
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

Expected output:

If you're already logged in:
```
    Already logged in as xxxxx
    Subscription set to xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
```
If you aren't logged in a browser window will pop-up which allows you to log in
```
    Logging in...
    Subscription set to xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
```

## Step 2:   Define project unifier

The project unifier is used to allow multiple deployments of services which have a need for a unique custom endpoint.

In [ ]:
$random = Get-Random -Minimum 100 -Maximum 999

Write-Host "Unifier set to: $random"

Expected output:

```
Unifier set to: xxx
```

## Step 3:   Create Resource Group

In this sample all resources are deployed to `eastus`. Feel free to change to your preferred location.

In [ ]:
$resourceGroup = "OpenAI-DataPro-RG"
$location = "eastus"

az group create `
    --location $location `
    --resource-group $resourceGroup

Expected output: 

 `JSON string describing the newly created resource group`

## Step 4:   Create Azure OpenAI instance

An instance of Azure Cognitive Service with the kind `OpenAI` will be created. The `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
$csOpenAIName = "aiservices$random"

az cognitiveservices account create `
    --name $csOpenAIName `
    --resource-group $resourceGroup `
    --location $location `
    --kind OpenAI `
    --sku S0 `
    --yes

$csOpenAIEndpoint = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query properties.endpoint `
        --output tsv `
)

$csOpenAIApiKey = (
    az cognitiveservices account keys list `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query key1 `
        --output tsv `
)

Expected output: 

 `JSON string describing the newly created Azure OpenAI instance`

## Step 5:   Deploy Azure OpenAI models

Two LLM models are deployed to the newly created Azure Cognitive Service instance: 

- A chat completion model. In the sample we're deploying `gpt-35-turbo`. This can be replaced with other models providing a chat completion interface like `gpt-4`.
- A text embedding model. In the sample we're deploying `text-embedding-ada-002`. Any other text embedding model can be deployed as well.

In [ ]:
# Chat Completion Model GPT-3.5-turbo
$AOAI_GPT35_DEPLOYMENT = "gpt-35-turbo"
$modelName = "gpt-35-turbo"
$modelVersion = "0301"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_GPT35_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# Chat Completion Model GPT-4
$AOAI_GPT4_DEPLOYMENT = "gpt-4"
$modelName = "gpt-4"
$modelVersion = "0613"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_GPT4_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# Text Embedding Model
$AOAI_EMBEDDING_DEPLOYMENT = "text-embedding-ada-002"
$modelName = "text-embedding-ada-002"
$modelVersion = "2"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_EMBEDDING_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# GPT-4 Vision Model - GPT-4 Turbo with Vision is the version of GPT-4 that accepts image inputs. It is available as the vision-preview model of gpt-4. The vision-preview model is available in only certain regions. For more information, see
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#standard-deployment-model-availability
$AOAI_GPT4VISION_DEPLOYMENT = "gpt-4-vision"
$modelName = "gpt-4"
$modelVersion = "vision-preview"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_GPT4VISION_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

Expected output: 

 `JSON string describing the newly deployed models`

## Step 6:   Create Azure Cognitive Search

Azure Cognitive Search is deployed to use its [vector DB functionalities](https://learn.microsoft.com/en-us/azure/search/vector-search-overview). Just like with Azure OpenAI Cognitive Service, the `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
$csSearchName = "aisearch$random"
$csSearchSku = "standard"

az search service create `
    --name $csSearchName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $csSearchSku

$csSearchEndpoint = "https://$csSearchName.search.windows.net"

$csSearchApiKey = ( `
    az search admin-key show `
        --resource-group $resourceGroup `
        --service-name $csSearchName `
        --query primaryKey `
        --output tsv `
)

Expected output: 

 `JSON string describing the newly created cognitive search resource`

## Step 7:   Create Azure Storage account

Azure Storage is deployed to store data that can be used to generate indexes in Azure Cognitive Search.

In [ ]:
$stgName = "aistg$random"
$stgSku = "Standard_LRS"

az storage account create `
    --name $stgName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $stgSku `
    --kind StorageV2 `
    --https-only true `
    --access-tier Hot

$stgConnectionString = ( `
    az storage account show-connection-string `
        --name $stgName `
        --resource-group $resourceGroup `
        --query connectionString `
        --output tsv `
)

Expected output: 

`JSON string describing the newly created storage account`

## Step 8: Set environment variables & create application.env file

In [ ]:
# Set environment variables

$ENV:AOAI_ENDPOINT = $csOpenAIEndpoint
$ENV:AOAI_APIKEY = $csOpenAIApiKey

#Azure Open AI GPT 3.5
$ENV:AOAI_GPT35_DEPLOYMENT = $AOAI_GPT35_DEPLOYMENT

#Azure Open AI Embedding - text-embedding-ada-002
$ENV:AOAI_EMBEDDING_DEPLOYMENT = $AOAI_EMBEDDING_DEPLOYMENT

#Azure Open AI GPT 4
$ENV:AOAI_GPT4_DEPLOYMENT = $AOAI_GPT4_DEPLOYMENT

#Azure Open AI GPT 4 Vision
$ENV:AOAI_VISION_ENDPOINT = $csOpenAIEndpoint
$ENV:AOAI_VISION_APIKEY = $csOpenAIApiKey
$ENV:AOAI_GPT4VISION_DEPLOYMENT = $AOAI_GPT4VISION_DEPLOYMENT

# Azure Search
$ENV:SEARCH_ENDPOINT = "https://$csSearchEndpoint/"
$ENV:SEARCH_APIKEY = $csSearchApiKey

$ENV:STORAGE_CONNECTIONSTRING = $stgConnectionString
$ENV:ASSET_FOLDER = "../../../../assets"

Write-Host "Environment variables set!"

$configurationFile = "../.env"

function Set-ConfigurationFileVariable($configurationFile, $variableName, $variableValue) {
    if (Select-String -Path $configurationFile -Pattern $variableName) {
        (Get-Content $configurationFile) | Foreach-Object {
            $_ -replace "$variableName = .*", "$variableName=$variableValue"
        } | Set-Content $configurationFile
    } else {
        Add-Content -Path $configurationFile -value "$variableName=$variableValue"
    }
}

Set-ConfigurationFileVariable $configurationFile "AOAI_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "AOAI_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "AOAI_GPT35_DEPLOYMENT" $AOAI_GPT35_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "AOAI_EMBEDDING_DEPLOYMENT" $AOAI_EMBEDDING_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "AOAI_GPT4_DEPLOYMENT" $AOAI_GPT4_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "AOAI_VISION_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "AOAI_VISION_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "AOAI_GPT4VISION_DEPLOYMENT" $AOAI_GPT4VISION_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "SEARCH_ENDPOINT" $csSearchEndpoint
Set-ConfigurationFileVariable $configurationFile "SEARCH_APIKEY" $csSearchApiKey
Set-ConfigurationFileVariable $configurationFile "STORAGE_CONNECTIONSTRING" $stgConnectionString
Set-ConfigurationFileVariable $configurationFile "ASSET_FOLDER" "../../../../assets"


Write-Host "Configuration file created at: $configurationFile"

Expected output:

```
Environment variables set!
Configuration file created at: xxxxxxxxxxxx
```

# 3.2 Basic Chat

In this notebook, we'll explore basic prompt engineering techniques and recommendations that will help us elicit responses from Azure OpenAI Models

### Visual Studio Code

If you are running these steps below in Visual Studio Code make sure you switch your kernal to Python


In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install openai
%pip install panel 

In [ ]:
%pip show openai

In [ ]:
import os
from openai import AzureOpenAI

print(os.getenv("AOAI_ENDPOINT"))
print(os.getenv("AOAI_APIKEY"))

client = AzureOpenAI(
  azure_endpoint = os.getenv("AOAI_ENDPOINT"),
  api_key = os.getenv("AOAI_APIKEY"),  
  api_version = "2023-07-01-preview"
)

chatgpt_model_name = os.getenv("AOAI_GPT4_DEPLOYMENT")
print(chatgpt_model_name)

Chat models take a series of messages as input, and return a model-generated message as output. The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).

In [ ]:
# A sample API call for chat completions looks as follows:
# Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).
# For more info: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions
import openai
try:
   
    response = client.chat.completions.create(
    model=chatgpt_model_name,
    messages=[
        {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
        {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ],
    temperature=0,
    max_tokens=800
    )

    print(response.choices[0].message)

 
except openai.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.APITimeoutError as e:
    # Handle request timeout
    print(f"Request timed out: {e}")
    
except:
    # Handles all other exceptions
    print("An exception has occured.")

## C# Example:  Setup Parameters

In [ ]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json, 7.0.3"
#r "nuget: Newtonsoft.Json, 13.0.1"
using DotNetEnv;

using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;

static string _configurationFile = @"../.env";
Env.Load(_configurationFile);

string apiBase = Environment.GetEnvironmentVariable("AOAI_ENDPOINT"); 
string apiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY"); 
string deploymentName = Environment.GetEnvironmentVariable("AOAI_GPT4_DEPLOYMENT"); 
string apiVersion = "2023-07-01-preview";

Expected output
```
Installed Packages
    DotNetEnv, 2.5.0
    Newtonsoft.Json, 13.0.1
    System.Text.Json, 7.0.3
```

In [ ]:
Console.WriteLine(apiBase);
Console.WriteLine(apiKey);
Console.WriteLine(deploymentName);


## Create completions for chat messages with GPT models

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

In [ ]:
var requestPayload = new JsonObject
{
    { "messages", new JsonArray
        {
            new JsonObject
            {
                { "role", "system" },
                { "content", "You are an AI assistance who You extract intention from provided text. You always answer with intention:" }
                
            },
            new JsonObject
            {
                { "role", "user" },
                { "content", "I'm not receiving calls on my Samsung Galaxy S22. Can you help?" }
            }
        }
    },
    { "max_tokens", 200 },
    { "temperature", 0.7 },
    { "frequency_penalty", 0 },
    { "presence_penalty", 0 },
    { "top_p", 0.95 },
    { "stop", null }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true // Optional: to make the JSON string more readable
});

        
string endpoint = $"{apiBase}openai/deployments/{deploymentName}/chat/completions?api-version={apiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("api-key",apiKey);
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var response = await httpClient.PostAsync(endpoint, stringContent);

    if (response.IsSuccessStatusCode)
    {
        using (var responseStream = await response.Content.ReadAsStreamAsync())
        {
            // Parse the JSON response using JsonDocument
            using (var jsonDoc = await JsonDocument.ParseAsync(responseStream))
            {
                // Access the message content dynamically
                var root = jsonDoc.RootElement;
                var messageContent = root.GetProperty("choices")[0].GetProperty("message").GetProperty("content").GetString();

                // Output the message content
                Console.WriteLine("Output: " + messageContent);
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {response}");
    }
}

Expected output:

```
Output: Intention: Requesting technical assistance with phone call issue on Samsung Galaxy S22.
```

# 3.3 Tokenization

In this notebook, we'll explore basic concepts behind tokenization, how to use the Microsoft.ML.Tokenizers library to tokenize text and get information about token counts

https://github.com/Azure-Samples/openai-dotnet-samples/blob/main/tokenization.ipynb

### Install Microsoft.ML.Tokenizers

In [ ]:
#r "nuget:Microsoft.ML.Tokenizers"

Installed Packages
 - Microsoft.ML.Tokenizers, 0.21.1

### Add using statements

In [ ]:
using Microsoft.ML.Tokenizers;

### Download and define vocab resources

Download the following files and place them in the root directory. These vocabulary files are what are used to encode the text into tokens.

- [GPT Vocabulary Files](https://huggingface.co/gpt2/tree/main)
    - vocab.json
    - merges.txt

In [ ]:
var vocabFilePath = @"../assets/vocab.json";
var mergeFilePath = @"../assets/merges.txt";

### Initialize Tokenizer

In [ ]:
var tokenizer = new Tokenizer(new Bpe(vocabFilePath, mergeFilePath));

### Encode text into tokens

In [ ]:
var input = "the brown fox jumped over the lazy dog!";
var tokenizerEncodedResult = tokenizer.Encode(input);
tokenizerEncodedResult

### Get token count

In [ ]:
tokenizerEncodedResult.Tokens.Count()

# 3.3 Prompts & Completions

In this section, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [ ]:
%pip install openai
%pip install panel 

In [ ]:
import os
from openai import AzureOpenAI

print(os.getenv("AOAI_ENDPOINT"))
print(os.getenv("AOAI_APIKEY"))

client = AzureOpenAI(
  azure_endpoint = os.getenv("AOAI_ENDPOINT"),
  api_key = os.getenv("AOAI_APIKEY"),  
  api_version = "2023-07-01-preview"
)

# Formating the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot - which refers to providing no examples
* One-Shot.
* Few-Shots - The term few-shot refers to providing a few of examples to help the model learn what it needs to do

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.


In [ ]:
#Functio to call the model.
def return_OAIResponse(user_message, context):

#As we can see, we’re adding the user’s question at the end of the prompt with the user role, so the model understands that this is a user request and not an instruction on how it should work.
    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    # print(newcontext)

    response = client.chat.completions.create(
        model="gpt-35-turbo", # model = "deployment_name".
        messages=newcontext,
        temperature=0,
        max_tokens=800
    )

    # print(response)
    print(response.model_dump_json(indent=2))

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [ ]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

In [ ]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [ ]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Bettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

In [ ]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [ ]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Bettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [ ]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answew the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Bettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

### Few Shots for classification.

In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Setiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

In [ ]:
context_user=[
        {"role": "system", "content": "You are an OrderBot in a fastfood restaurant."},
        {"role": "user", "content": "I have only 10 dollars, what can I order?"},
        {"role": "assistant", "content": "We have the fast menu for 7 dollars."},
        {"role": "user", "content": "Perfect! Give me one! "}
]
print(return_OAIResponse("", context_user))

## Content Generation

In this section, we'll explore how to use LLMs to do content generation

In [ ]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [ ]:
def mock_get_car() -> dict:
    return {
        "make": "Ford",
        "model": "Explorer",
        "base": "XLT",
        "color": "red",
        "year": 2019,
        "condition": "good",
        "mileage": 32000,	
        "price": 25000
    }

def get_car_description(car: dict) -> str:
    return f'{car["year"]} {car["make"]} {car["model"]} {car["base"]} {car["color"]} with {car["mileage"]} miles in {car["condition"]} condition for ${car["price"]}.'

car = mock_get_car()
car_description = get_car_description(car)

# Create a semantic kernel inline function
sales_desc_generation_template = "Create a one paragraph sales description that includes the price for a {{input}}"
template = common.render_template(sales_desc_generation_template, input=car_description)

# Execute the SK function
print(common.Call_OpenAI(client,common.gpt_api_deployment,template,max_tokens=500))

## Classification

In this section, we'll explore how to use LLMs to do classification

In [ ]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version="2023-12-01-preview",
        azure_endpoint=common.api_URI)

print(common.api_URI)

In [ ]:
prompt_template = """For the following list of animals:

- Dog
- Cat
- Elephant
- Dolphin
- Shark
- Whale
- Snake

Can you classify and list by animal type?
"""

result = common.Call_OpenAI(client,common.gpt_api_deployment,prompt_template)

print(result)

## Recommendations

In this section, we'll explore how to use LLMs to do recommendations

In [ ]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [ ]:
def mock_get_restaurant_list(cityCode) -> list[str]:
    if (cityCode == "MIA"):
        return [
            "Joe's Stone Crab",
            "Versailles",
            "Hillstone",
            "Casa Tua",	
            "Cecconi's",
            "Yardbird Southern Table & Bar",
        ]
    return []

target_text=""
for restaurant in mock_get_restaurant_list("MIA"):
    target_text += f"{restaurant}\n"

print(target_text)

recommendation_template = 'List two top restaurants:\n{{input}}\nOut in JSON format.'
print(recommendation_template)
rendered_template = common.render_template(recommendation_template, input=target_text)
print(rendered_template)

print(common.Call_OpenAI(client,common.gpt_api_deployment,rendered_template,max_tokens=200))

## Translation with Semantic Kernal

In this section, we'll explore how to use LLMs to do recommendations

In [ ]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

print(common.api_KEY)
print(common.api_version)
print(common.api_URI)
# print(azure_endpoint)

In [ ]:
import common

# Get a configured Semantic Kernel
# Note all other demos except this one use the OpenAI SDK
kernel = common.get_kernel()